# DAG

Дан ориентированный граф. 

1) Нужно определить, есть ли в графе цикл
2) Если цикл есть -- вывести его (достаточно одного цикла)
3) Если цикла нет -- применяем топологическую сортировку и выводим результат

Подразумевается, что граф подается на вход в виде списка смежности (словарь со списками ребер). 

Тесты. 

https://www.youtube.com/watch?v=pB83tSvoBuc

# Import

In [91]:
import os

while os.getcwd().split("/")[-1] != "algorithms_python":
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [92]:
from collections import deque
from typing import Dict, List, Tuple, Any

import numpy as np

# detect and print cycle in dag

In [98]:
def find_cycle_dag(graph: Dict[Any, List[Any]]) -> Tuple[bool, List[Any]]:
    # graph keys to numbers
    vertices = list(graph.keys())
    vertex_to_idx = {vertice: i for i, vertice in enumerate(vertices)}
    idx_to_vertex = {i: vertice for i, vertice in enumerate(vertices)}

    customized_graph = dict()
    for vertex, neighbors in graph.items():
        customized_graph[vertex_to_idx[vertex]] = []
        for neighbor in neighbors:
            customized_graph[vertex_to_idx[vertex]].append(vertex_to_idx[neighbor])

    # Find cycle

    # 0 - не посещена, 1 - в обработке, 2 - обработана
    state = [0] * len(vertices)
    parent = [-1] * len(vertices)
    cycle = []

    def dfs_find_cycle(customized_graph, vertice, state, parent, cycle):
        state[vertice] = 1

        for neighbor in customized_graph[vertice]:
            if not cycle:
                if state[neighbor] == 0:
                    parent[neighbor] = vertice

                    if dfs_find_cycle(customized_graph, neighbor, state, parent, cycle):
                        return True
                elif state[neighbor] == 1:
                    current = vertice
                    while current != neighbor:
                        cycle.append(current)
                        current = parent[current]
                    cycle.append(neighbor)
                    cycle.append(vertice)
                    cycle.reverse()
                    return True

        state[vertice] = 2
        return False

    for i in range(len(vertices)):
        if state[i] == 0 and not cycle:
            dfs_find_cycle(customized_graph, i, state, parent, cycle)

    if cycle:
        cycle_vertices = [idx_to_vertex[i] for i in cycle]
        return True, cycle_vertices

    # topological_sort
    visited = [False] * len(vertices)
    topological_order = []

    def dfs_topological_sort(customized_graph, vertice, visited, topological_order):
        visited[vertice] = True
        for neighbor in customized_graph[vertice]:
            if not visited[neighbor]:
                dfs_topological_sort(
                    customized_graph, neighbor, visited, topological_order
                )
        topological_order.append(vertice)

    for i in range(len(vertices)):
        if not visited[i]:
            dfs_topological_sort(customized_graph, i, visited, topological_order)

    topological_order.reverse()
    topological_vertices = [idx_to_vertex[i] for i in topological_order]
    return False, topological_vertices

In [99]:
graph = {
    "A": ["D"],
    "B": [],
    "C": ["B"],
    "D": ["B", "C"]
}
find_cycle_dag(graph)

(False, ['A', 'D', 'C', 'B'])

In [100]:
graph = {
    0: [1],
    1: [2],
    2: [0, 3],
    3: []
}
find_cycle_dag(graph)

(True, [2, 0, 1, 2])

In [101]:
graph = {
    0: [1, 2],
    1: [2, 3],
    2: [0, 3],
    3: []
}
find_cycle_dag(graph)

(True, [2, 0, 1, 2])